In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/nlp-getting-started/train.csv
/kaggle/input/nlp-getting-started/test.csv


In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression, Ridge
import matplotlib.pyplot as plt
import re
from sklearn.linear_model import LinearRegression, Ridge, Lasso, BayesianRidge
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn import preprocessing
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.neural_network import MLPRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.linear_model import RidgeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier
from sklearn.datasets import make_classification
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from catboost import Pool
from gensim.utils import tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, LabelEncoder, MultiLabelBinarizer
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline

In [ ]:
import nltk
stemmer = SnowballStemmer('english')
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

In [ ]:
test = pd.read_csv("../input/nlp-getting-started/test.csv")
train = pd.read_csv("../input/nlp-getting-started/train.csv")
submission = pd.read_csv("../input/nlp-getting-started/sample_submission.csv")

Let's make preprocessing and vectorizing texts;
We will use TF-IDF and usual LogisticRegression

In [ ]:


def remove_urls(vTEXT):
    vTEXT = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', vTEXT, flags=re.MULTILINE)
    return(vTEXT)


# def 
def preprocessing(df):
    text_list = list(df['text'])
    preprocessed_texts = []
    for i in range(len(text_list)):
        text_tokenized = remove_urls(text_list[i])
        text_tokenized = list(tokenize(text_tokenized, lower = True, deacc = True))
#         preprocessed_texts.append(list(text_tokenized))
        text_stemmed = [stemmer.stem(w) for w in text_tokenized if w not in stop_words]
        text_without_stop_words = [w for w in text_stemmed if w not in stop_words]
        preprocessed_texts.append(text_without_stop_words)
        
    for i in range(len(preprocessed_texts)):
        preprocessed_texts[i] = " ".join(preprocessed_texts[i])
    return preprocessed_texts  #возвращает корпус обработанных твитов



def model_vect_texts(df):
    vectorizer = TfidfVectorizer() #модель для векторизации текстов
    preprocessed_texts_for_train = preprocessing(df) #на каком корпусе текстов будем обучать vectorizer
    vectorizer.fit(preprocessed_texts_for_train) #обучаем vectorizer
    return vectorizer


def preprocessing_features(df, vectorizer):
    preprocessed_texts = preprocessing(df)

    tfidf = vectorizer.transform(preprocessed_texts)
#     df_tfidf = pd.DataFrame(tfidf.toarray(), columns = list(vectorizer.get_feature_names()))
#     df_tfidf['loc'] = df['location'] #их слишком много, они практичсеки эквивалентны 'id' как и keyword
#     df_tfidf = pd.get_dummies(df_tfidf)
    
    return tfidf

In [ ]:
X = pd.DataFrame(train)
X_train, X_test, y_train, y_test = train_test_split(X, train['target'])
vectorizer_train = model_vect_texts(X_train)
X_train = preprocessing_features(X_train, vectorizer_train)
X_test = preprocessing_features(X_test, vectorizer_train)
# X_train = vectorizer_train.transform(X_train)
# X_test = vectorizer_train.transform(X_test)

In [ ]:
model = LogisticRegression()
model.fit(X_train, y_train)
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)
print(f'LogisticRegression on train, f1_score: {f1_score(y_pred_train, y_train)}')
print(f'LogisticRegression on test, f1_score: {f1_score(y_pred_test, y_test)}')



let's train our model on the final DataFarme:


In [ ]:
vectorizer_final = model_vect_texts(train)
y = train['target']
train_final = preprocessing_features(train, vectorizer_final)
test_final = preprocessing_features(test, vectorizer_final)

In [ ]:
model = LogisticRegression()
model.fit(train_final, y)
y_pred_train_final = model.predict(train_final)
y_pred_test_final = model.predict(test_final)
print(f'LogisticRegression on train, f1_score: {f1_score(y_pred_train_final, y)}')
y_pred_test_final


